# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from scipy.ndimage import affine_transform
from skimage import measure
import SimpleITK as sitk
import matplotlib.pyplot as plt


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation, affine_transform_volume
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [ ]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

def load_ants_affine_mat(mat_file_path):
    # Load .mat file

    rotation = np.array([
    [0.941498875617981, 0.19202075898647308, -0.08786671608686447], 
    [-0.198630228638649, 0.781607985496521, 0.015325229614973068],
    [0.06243732571601868, 0.0008880351670086384, 0.8622015118598938]
    ]
    )
    translation = np.array([266.45843505859375, 150.64747619628906, -138.71780395507812])
    center = np.array([730.082275390625, 373.0629577636719, 569.5])
    # Apply rotation around center
    affine = np.eye(4)
    affine[:3, :3] = rotation
    affine[:3, 3] = translation + center - rotation @ center

    return affine

def convert_lps_to_sar(affine_lps):
    # Flip L (x) and P (y) axes: LPS to RAS => flip x and y
    flip = np.diag([-1, -1, 1, 1])
    affine_sar = flip @ affine_lps @ flip
    return affine_sar

In [ ]:
moving_name = 'MD594'
fixed_name = 'Allen'
moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = list_coms(fixed_name, scaling_factor=10)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
#bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

In [ ]:
from scipy.io import loadmat
reg_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
transform_file = 'MD594_Allen_10.0x10.0x10.0um_Affine_inverse.mat'
transform_path = os.path.join(reg_path, moving_name, transform_file)
transform = sitk.ReadTransform(transform_path)
transform = sitk.AffineTransform(transform)
print(transform)

In [ ]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

In [ ]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS 238.5831606646421
# MD585 to MD589 RMS 18.2658167690059

In [ ]:
transformed_elastix = {'10N_L': (1245.825565, 583.011918, 541.248353), '10N_R': (1241.078327, 575.816747, 604.932277), '12N': (1234.494263, 599.28526, 571.130307), '3N_L': (916.08634, 393.30332, 556.006873), '3N_R': (915.61113, 395.207295, 587.248926), '4N_L': (954.697061, 397.285593, 544.243496), '4N_R': (955.039712, 398.093118, 598.067921), '5N_L': (1018.137495, 520.533489, 409.881654), '5N_R': (1019.341376, 525.263033, 727.129661), '6N_L': (1068.668552, 527.691811, 539.596871), '6N_R': (1073.775168, 536.06989, 613.966415), '7N_L': (1083.300099, 645.703034, 437.587277), '7N_R': (1087.207846, 667.340715, 701.69945), '7n_L': (1045.956409, 566.507496, 447.340355), '7n_R': (1054.223742, 570.088494, 689.160729), 'AP': (1244.090532, 552.686906, 571.760582), 'Amb_L': (1160.638022, 636.809303, 427.495323), 'Amb_R': (1165.984467, 653.856579, 711.948937), 'DC_L': (1123.751951, 469.786863, 315.849668), 'DC_R': (1131.744401, 461.873771, 812.028033), 'IC': (1034.298923, 232.223501, 566.79765), 'LC_L': (1045.189532, 448.35284, 475.846568), 'LC_R': (1050.580737, 454.508383, 663.837542), 'LRt_L': (1233.211435, 702.321565, 443.053938), 'LRt_R': (1234.112904, 699.030951, 697.872942), 'PBG_L': (956.449566, 360.380351, 354.569236), 'PBG_R': (958.509989, 371.76958, 788.302734), 'Pn_L': (895.19021, 624.89548, 501.952816), 'Pn_R': (894.018076, 634.206201, 645.330084), 'RMC_L': (888.025505, 450.25689, 511.06366), 'RMC_R': (884.667685, 457.312469, 622.186793), 'RtTg': (940.155521, 592.96321, 566.352029), 'SC': (915.771574, 229.443045, 566.999621), 'SNC_L': (835.409636, 512.465283, 429.363112), 'SNC_R': (828.798088, 507.753302, 711.989699), 'SNR_L': (865.722722, 506.533869, 407.481066), 'SNR_R': (857.665458, 509.930588, 719.266855), 'Sp5C_L': (1301.972326, 627.651522, 413.166233), 'Sp5C_R': (1298.198967, 619.19027, 728.645978), 'Sp5I_L': (1222.916041, 590.080036, 364.984897), 'Sp5I_R': (1235.268962, 588.638673, 774.199137), 'Sp5O_L': (1141.028347, 577.329771, 364.832388), 'Sp5O_R': (1151.739201, 583.391376, 768.576876), 'Tz_L': (983.843401, 656.759861, 508.510555), 'Tz_R': (986.412422, 666.944614, 626.51371), 'VCA_L': (1048.046155, 509.507162, 297.514027), 'VCA_R': (1061.438954, 507.154848, 860.320689), 'VCP_L': (1095.838599, 548.173446, 299.199061), 'VCP_R': (1115.532484, 526.161194, 846.116132), 'VLL_L': (938.78711, 555.30814, 396.500515), 'VLL_R': (940.407198, 557.855505, 737.535395)}


In [ ]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    transformed = transformed_elastix[structure]
    fixed0 = np.array(fixed_all[structure]) 
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))


In [ ]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = affine_transform_point(com, transformation_matrix)
x,y,z = com

print(f'{moving_name} {structure} apply ants {x=}, {y=} {z=}')

print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')
elastix_com = np.array([915, 230, 568])
diff = elastix_com - np.array(fixed_all[structure])
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')

In [ ]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

In [ ]:
outpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/com'
for structure, com in moving_all.items():
    comfile = structure + '.txt'
    compath = os.path.join(outpath, comfile)
    np.savetxt(compath, com)

In [ ]:
um = 50
registration_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
base_com_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
for brain in [moving_name, fixed_name]:
    brain_point_path = os.path.join(registration_path, brain, f'{brain}_{um}um_sagittal.pts')
    brain_com_path = os.path.join(base_com_path, brain, 'com')
    comfiles = sorted(os.listdir(brain_com_path))
    with open(brain_point_path, 'w') as f:
        f.write('point\n')
        f.write(f'{len(common_keys)}\n')
        for comfile in comfiles:
            structure = comfile.replace('.txt','')
            if structure in common_keys:
                #print(structure)
                compath = os.path.join(brain_com_path, comfile)
                x,y,z = np.loadtxt(compath)
                f.write(f'{round(x/um,4)} {round(y/um,4)} {round(z/um,4)}')
                f.write('\n')


In [ ]:
def ants_3d_to_scipy_2d(affine_3d, plane='axial', slice_index=0):
    """
    Convert a 3D ANTs affine transformation matrix to a 2D affine transform
    suitable for scipy.ndimage.affine_transform.

    Parameters:
        affine_3d (np.ndarray): A 4x4 affine matrix from ANTs.
        plane (str): Plane to slice through ('axial', 'coronal', 'sagittal').
        slice_index (int): Index of the slice in the chosen plane.

    Returns:
        matrix_2d (np.ndarray): 2x2 affine transformation matrix.
        offset_2d (np.ndarray): Length-2 offset vector.
    """
    if affine_3d.shape != (4, 4):
        raise ValueError("Expected a 4x4 affine transformation matrix.")

    # Extract rotation+scaling and translation components
    rotation_scaling = affine_3d[:3, :3]
    translation = affine_3d[:3, 3]

    if plane == 'axial':
        matrix_2d = rotation_scaling[:2, :2]
        offset_2d = translation[:2] + rotation_scaling[:2, 2] * slice_index
    elif plane == 'coronal':
        matrix_2d = rotation_scaling[[0,2], :][:, [0,2]]
        offset_2d = translation[[0,2]] + rotation_scaling[[0,2], 1] * slice_index
    elif plane == 'sagittal':
        matrix_2d = rotation_scaling[1:3, 1:3]
        offset_2d = translation[1:3] + rotation_scaling[1:3, 0] * slice_index
    else:
        raise ValueError("Plane must be 'axial', 'coronal', or 'sagittal'.")

    return matrix_2d, offset_2d

In [ ]:
reg_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
matrix_path = os.path.join(reg_path, 'ALLEN771602/ALLEN771602_Allen_32.0x28.8x28.8um_sagittal.tfm')
transform = sitk.ReadTransform(matrix_path)

In [ ]:
tx = sitk.AffineTransform(transform)
Rt = tx.GetMatrix()
R = np.array(Rt).reshape(3,3)
print(R)
print()
Rr = np.rot90(R, k=1)
print(Rr)

In [ ]:
affine_transform = sitk.AffineTransform(3)
affine_transform.SetMatrix(Rr.flatten())
affine_transform.SetTranslation(t)
affine_transform.SetCenter(c)


In [ ]:
# Example 3x3 array where each row is a 3D point
A = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

# Rotation matrix for 90 degrees around Y-axis (counter-clockwise)
theta = np.radians(90)
rotation_matrix = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0,            1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])

# Apply rotation
rotated = A @ rotation_matrix.T
print(rotated)

In [ ]:
rotation_matrix.T